In [ ]:
-- In viz/dashboard.ipynb via pandas.read_sql_query
WITH cdi_md AS (
  SELECT yearstart AS year, topic, question, datavalue
  FROM cdi
  WHERE locationabbr = 'MD'
    AND topic = 'Diabetes'
    AND datavaluetype = 'Crude prevalence'
),
readmit AS (
  SELECT CAST(strftime('%Y', date(date_key)) AS INT) AS year,
         AVG(CASE WHEN was_readmit='True' THEN 1.0 ELSE 0.0 END) AS readmit_rate
  FROM encounters
  GROUP BY year
)
SELECT r.year, r.readmit_rate, c.datavalue AS diabetes_prevalence_pct
FROM readmit r
LEFT JOIN cdi_md c ON c.year = r.year
ORDER BY r.year;


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [ipywidgets]3 [ipywidgets]

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip
usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

options:
  -h, --help     show this help message and exit
  --version      show the versions of core jupyter packages and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable
                 format.
  --json         output paths as machine-readable json
  --debug        output debug information about paths

Available subcommands: 